In [1]:
import json
import re
from collections import defaultdict
import numpy as np

In [2]:
def parse_side_interest(side_str):
    species = {}
    for term in side_str.split('+'):
        term = term.strip()
        m = re.match(r'(\d*)\s*([A-Za-z0-9_]+)', term)
        if not m:
            raise ValueError(f"cannot parse '{term}'")
        
        name = m.group(2)
        name_split = name.split("_")
        if len(name_split) != 2:
            continue
        
        print(name)
        
        coeff = int(m.group(1)) if m.group(1) else 1
        species[name] = coeff
    return species


def parse_side(side_str):
    species = {}
    for term in side_str.split('+'):
        term = term.strip()
        m = re.match(r'(\d*)\s*([A-Za-z0-9_]+)', term)
        if not m:
            raise ValueError(f"cannot parse '{term}'")
        
        
        coeff = int(m.group(1)) if m.group(1) else 1
        name  = m.group(2)
        species[name] = coeff
    return species


def load_reactions(path):
    with open(path) as f:
        raw = json.load(f)
    reactions = []
    all_species = set()
    for r in raw:
        left, right = r['equation'].split('<->') if '<->' in r['equation'] else r['equation'].split('->')
        rev = '<->' in r['equation']
        left  = parse_side_interest(left)
        right = parse_side_interest(right)
        
        # print("left: ", left.keys())
        # print("right: ", right.keys())
        
        all_species |= left.keys() | right.keys()
        reactions.append({
            'id':         r['id'],
            'left':     left,
            'right':    right,
            'rate':     r['rate']
        })
    species_list = sorted(all_species)
    return reactions, species_list

In [3]:

reactions, species_list = load_reactions("reactions.json")

V_F
O_F
O_F
V_F
V_S
O_S
O_F
V_F
O_F
V_S
V_F
O_S
O_S
O_F
O_F
O_S
V_F
V_S
O_F
O_F
V_F
V_F


In [4]:
print(reactions)
print(species_list)

[{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption'}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption'}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption'}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb-ER'}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'right': {'V_F': 1, 'O_S': 1}, 'rate': 'diffusion'}, {'id': 6, 'left': {'O_S': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption'}, {'id': 7, 'left': {'O_F': 1, 'O_S': 1}, 'right': {'V_F': 1, 'V_S': 1}, 'rate': 'diffusion-recomb'}, {'id': 8, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb-LH'}]
['O_F', 'O_S', 'V_F', 'V_S']


In [5]:
### conservation laws

surface_species = [ele.split('_') for ele in species_list]

surface_species_unique = list(set([ele[1] for ele in surface_species]))

dictionaires_species = {}

for group in surface_species_unique:
    vec_species = []
    for specie in surface_species:
        if group == specie[1] and specie[0] != "V":
            vec_species.append("_".join(specie))
        
    dictionaires_species["V_" + group] = vec_species



print(surface_species)
print(surface_species_unique)
print(dictionaires_species)

[['O', 'F'], ['O', 'S'], ['V', 'F'], ['V', 'S']]
['F', 'S']
{'V_F': ['O_F'], 'V_S': ['O_S']}


In [6]:

#### compose the matrix S

effective_species_list = np.array(list(dictionaires_species.values())).squeeze()
print(effective_species_list)



['O_F' 'O_S']


In [7]:
print(reactions)

[{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption'}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption'}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption'}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb-ER'}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'right': {'V_F': 1, 'O_S': 1}, 'rate': 'diffusion'}, {'id': 6, 'left': {'O_S': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption'}, {'id': 7, 'left': {'O_F': 1, 'O_S': 1}, 'right': {'V_F': 1, 'V_S': 1}, 'rate': 'diffusion-recomb'}, {'id': 8, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb-LH'}]


In [8]:


#### filter reactions for each specie and do the balaance for it 

print(effective_species_list)
filtered_reactions = []


for specie in effective_species_list:
    
    vec_aux = []
    for idx, eq in enumerate(reactions):
        
        # print(eq['left'])
        # print(eq['right'])
        
        if specie in eq["left"].keys():
            vec_aux.append((idx, -1))
        
        if specie in eq["right"].keys():
            vec_aux.append((idx, +1))
        
    filtered_reactions.append(vec_aux)


S_constant = []
S_linear = []
S_quadratic = []


print(filtered_reactions)

for idx, system in enumerate(filtered_reactions):
    
    print("system", system)
    
    S_constant_aux = []
    S_linear_aux = []
    S_quadratic_aux = []
    
    S_terms_aux = []
    
    for idx_eq, sign in system:

        term_keys = list(reactions[idx_eq]['left'].keys())
        term_nu = list(reactions[idx_eq]['left'].values())
        
        # print("term: ", term_keys)
        # print("term: ", term_nu)
        
        ### think about [(idx_1, idx_2, ..., idx_n, idx_reactiob, sigm), ....]
        
        if len(term_keys) == 1:
        
            if term_keys[0].split("_") == "V":
                S_terms_aux.append(([], idx_eq, sign))
                vec_aux = list(dictionaires_species[term_keys[0]])
                vec_tuple = [([effective_species_list.index(term_keys[0])], idx_eq, -1*sign)]
                
            else:
                ### only update the linear terms
                pass
        
        elif len(term['right'].keys()) == 2:
            
            ### if V is in : a scheme 
            
            ### if V is not in a different scheme 
            
            print("2: ", term['right'].keys())
        
        else:
            pass
        
    
    
print(effective_species_list)
print(filtered_reactions)







['O_F' 'O_S']
[[(0, 1), (1, -1), (3, -1), (4, -1), (5, 1), (6, -1), (7, -1)], [(2, 1), (4, 1), (5, -1), (6, -1)]]
system [(0, 1), (1, -1), (3, -1), (4, -1), (5, 1), (6, -1), (7, -1)]
V_F
O_F
O_F


NameError: name 'term' is not defined